### Create relevance metric (same for aml)


In [ ]:
%pip install mlflow==2.10.1
%pip install azureml-mlflow==1.55.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------- -------------- 0.7/1.0 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 16.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.6/4.8 MB 35.2 MB/s eta 0:00:01
   ------------- -------------------------- 1.7/4.8 MB 21.1 MB/s eta 0:00:01
   -------------------------- ------------- 3.2/4.8 MB 25.3 MB/s eta 0:00:01
   ---------------------------------------  4.8/4.8 MB 28.1 MB/s eta 0:00:01
   ---------------------------------------- 4.8/4.8 MB 25.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import mlflow
import openai
import os
import pandas as pd
from getpass import getpass
from azureml.core import Workspace

%reload_ext dotenv
%dotenv

subscription_id = os.environ["subscription_id"]
resource_group_name = os.environ["resource_group_name"]
workspace_name = os.environ["workspace_name"]

aoi_deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
azure_openai_key = os.environ["azure_openai_key"]
azure_aoai_endpoint = os.environ["azure_aoai_endpoint"]

os.environ.setdefault("OPENAI_API_KEY", azure_openai_key)
os.environ.setdefault("OPENAI_API_BASE", azure_aoai_endpoint)
os.environ.setdefault("OPENAI_API_VERSION", "2023-05-15")
os.environ.setdefault("OPENAI_API_TYPE", "azure")
os.environ.setdefault("OPENAI_DEPLOYMENT_NAME",aoi_deployment_name)

'chat'

### Create faithfulness metric (=aka groundedness for AML)


In [9]:
from mlflow.metrics.genai import faithfulness, EvaluationExample

# Create a good and bad example for faithfulness in the context of this problem
faithfulness_examples = [
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions. In Databricks, autologging is enabled by default. ",
        score=2,
        justification="The output provides a working solution, using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions.",
        score=5,
        justification="The output provides a solution that is using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
]

faithfulness_metric = faithfulness(model="openai:/gpt-4", examples=faithfulness_examples)

### Create relevance metric (same for aml)


In [10]:
from mlflow.metrics.genai import relevance, EvaluationExample

relevance_metric = relevance(model="openai:/gpt-4")
print(relevance_metric)

EvaluationMetric(name=relevance, greater_is_better=True, long_name=relevance, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's relevance based on the rubric
justification: Your reasoning about the model's relevance score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called relevance based on the input and output.
A definition of relevance and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Outpu

In [ ]:
# !az login --tenant fdpo.onmicrosoft.com
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(credential=DefaultAzureCredential(),
                        subscription_id=subscription_id, 
                        resource_group_name=resource_group_name,
                        workspace_name=workspace_name)


In [ ]:
mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
print(mlflow_tracking_uri)

azureml://swedencentral.api.azureml.ms/mlflow/v1.0/subscriptions/77e0e886-66a0-48ff-be94-66a9737fac63/resourceGroups/adnegrau-openai/providers/Microsoft.MachineLearningServices/workspaces/workshop-workspace-adina


In [6]:
ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group_name)

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [20]:
print(ws.get_mlflow_tracking_uri())

azureml://swedencentral.api.azureml.ms/mlflow/v2.0/subscriptions/77e0e886-66a0-48ff-be94-66a9737fac63/resourceGroups/adnegrau-openai/providers/Microsoft.MachineLearningServices/workspaces/workshop-workspace-adina?


Generate answers


In [36]:

with open("./output/qa/evaluation/solution-ops-200-qa-generated-answer", "r", encoding="utf-8") as file:
    qa_evluation_data = json.load(file)
    column_names = ["user_prompt", "output_prompt", "context"]
    final_df = pd.DataFrame(columns=column_names)
    final_df.from_dict(qa_evluation_data)
    # df = pd.DataFrame.from_records(map(json.loads, qa_evluation_data))
    try:
        results = mlflow.evaluate(data=final_df, predictions="generated_output", model_type="question-answering", extra_metrics=[faithfulness_metric, relevance_metric, mlflow.metrics.latency()],
        evaluator_config={
                "col_mapping": {
                    "inputs": "output_prompt", # Define the column name for the input
                    "context": "source",
                }
            })
        print(results)
    except Exception as e:
        print(e)

 Model registry functionality is unavailable; got unsupported URI 'azureml://swedencentral.api.azureml.ms/mlflow/v1.0/subscriptions/77e0e886-66a0-48ff-be94-66a9737fac63/resourceGroups/adnegrau-openai/providers/Microsoft.MachineLearningServices/workspaces/workshop-workspace-adina' for model registry data storage. Supported URI schemes are: ['', 'file', 'databricks', 'databricks-uc', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.


KeyboardInterrupt: 

In [ ]:
import openai

openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv(
    "AZURE_OPENAI_ENDPOINT"
)  # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"  # this might change in the future


def call_aoai_gpt4(messages: list[dict]):
    response = openai.ChatCompletion.create(
        engine=aoi_deployment_name, messages=messages  # engine = "deployment_name".
    )

    return response.choices[0].message.content

## Create experiment


In [3]:
experiment_name = "test-experiment"
mlflow.create_experiment(experiment_name)

'541817325950158082'

In [21]:
import json
import pandas as pd

# experiment_name = "test-experiment"
# mlflow.create_experiment(experiment_name, artifact_location="s3://your-bucket")

with open("./output/qa/evaluation/solution-ops-200-qa-generated-answer", "r", encoding="utf-8") as file:
    qa_evluation_data = json.load(file)
    df = pd.DataFrame.from_records(qa_evluation_data)
    # mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment("test-experiment")
    with mlflow.start_run(run_name="run1") as run:
        results = mlflow.evaluate(data=df, predictions="generated_output", model_type="question-answering", extra_metrics=[faithfulness_metric, relevance_metric, mlflow.metrics.latency()],
        evaluator_config={
                "col_mapping": {
                    "inputs": "output_prompt", # Define the column name for the input
                    "context": "source",
                }
            })

        # mlflow.log_metric('toxicity', results.metrics['toxicity/v1/p90'])
        mlflow.log_metric('faithfulness_mean', results.metrics['faithfulness/v1/mean'])

c:\Projects\Workshop2024\.venv\Lib\site-packages\mlflow\data\digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
c:\Projects\Workshop2024\.venv\Lib\site-packages\mlflow\models\evaluation\base.py:414: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/02/12 18:08:48 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/02/12 18:08:48 WARNING mlflow.models.evaluation.default_evaluator: Setting the latency to 0 for all entries because the model is not provided.
2024/02/12 18:08:48 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/02/12 18:08:48 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
20

In [22]:
results.metrics

{'latency/mean': 0.0,
 'latency/variance': 0.0,
 'latency/p90': 0.0,
 'faithfulness/v1/mean': 3.857142857142857,
 'faithfulness/v1/variance': 1.5510204081632657,
 'faithfulness/v1/p90': 5.0,
 'relevance/v1/mean': 3.4285714285714284,
 'relevance/v1/variance': 1.1020408163265307,
 'relevance/v1/p90': 4.0}